In [ ]:
!pip install selenium webdriver_manager beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 23.9 MB/s eta 0:00:00


In [ ]:
# Install Google Chrome Stable
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y # To fix any broken dependencies

--2025-12-29 01:50:52--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 172.217.204.190, 172.217.204.93, 172.217.204.136, ...
Connecting to dl.google.com (dl.google.com)|172.217.204.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117840328 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 112.38M   390MB/s    in 0.3s    

2025-12-29 01:50:52 (390 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [117840328/117840328]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (143.0.7499.169-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libatk-bridge2.0-0 (>= 2.5.

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def crawl_fsc_from_page(start_page=1):
    driver = None
    results = []

    try:
        # --- (브라우저 설정 옵션들은 그대로 유지) ---
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        options.add_argument(f'user-agent={user_agent}')

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""
        })
        driver.set_page_load_timeout(60)

        # 1. 초기 접속 (무조건 1페이지로 접속됨)
        url = "https://better.fsc.go.kr/fsc_new/replyCase/TotalReplyList.do?stNo=11&muNo=117&muGpNo=75"
        print(f"사이트 접속 중... (현재 1페이지)")
        driver.get(url)
        time.sleep(3)

        # 현재 브라우저의 페이지 상태 (1부터 시작)
        current_browser_page = 1

        while True:
            # ===============================================================
            # [추가된 로직] 목표 페이지(start_page)보다 작으면 무조건 '다음' 클릭해서 넘김
            # ===============================================================
            if current_browser_page < start_page:
                print(f" -> 현재 {current_browser_page}페이지... 목표({start_page}페이지)까지 이동 중...")

                try:
                    # '다음' 버튼 찾기
                    next_btn = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginate_button.next"))
                    )
                    # 다음 버튼 클릭
                    driver.execute_script("arguments[0].click();", next_btn)
                    current_browser_page += 1
                    time.sleep(1) # 이동 대기 (빠르게 넘기기 위해 시간 단축)
                    continue # 수집 로직 건너뛰고 바로 다음 루프로
                except Exception as e:
                    print(f"페이지 이동 중 에러 발생 (더 이상 페이지가 없을 수 있음): {e}")
                    break

            # ===============================================================
            # [수집 로직] 목표 페이지에 도달했을 때부터 수행
            # ===============================================================
            print(f"\n--- {current_browser_page} 페이지 수집 시작 ---")

            try:
                # 목록 로딩 확인
                try:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
                except:
                    print("목록 로딩 실패.")
                    break

                rows = driver.find_elements(By.CSS_SELECTOR, ".board-list tbody tr")
                # (빈 행 처리 로직 등은 기존과 동일)
                if len(rows) == 0:
                    rows = driver.find_elements(By.TAG_NAME, "tr")
                    rows = [r for r in rows if r.find_elements(By.TAG_NAME, "td")]

                post_count = len(rows)
                print(f"{current_browser_page}페이지: 총 {post_count}개의 게시물 발견")

                if post_count == 0:
                    break

            except Exception as e:
                print(f"목록 에러: {e}")
                break

            # --- (게시물 상세 수집 루프는 기존 코드와 동일하게 유지) ---
            for i in range(post_count):
                try:
                    # Stale 방지 재검색
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "tr")))
                    try: rows = driver.find_elements(By.CSS_SELECTOR, ".board-list tbody tr")
                    except:
                        rows = driver.find_elements(By.TAG_NAME, "tr")
                        rows = [r for r in rows if r.find_elements(By.TAG_NAME, "td")]

                    if i >= len(rows): continue
                    row = rows[i]

                    try:
                        title_link = row.find_element(By.TAG_NAME, "a")
                        title_text = title_link.text
                    except: continue

                    print(f" [{current_browser_page}P - {i+1}/{post_count}] {title_text}")

                    # 윈도우 핸들 저장 및 클릭
                    main_window = driver.current_window_handle
                    driver.execute_script("arguments[0].click();", title_link)
                    time.sleep(2)

                    # 새 창 처리
                    current_windows = driver.window_handles
                    is_new_window = False
                    if len(current_windows) > 1:
                        is_new_window = True
                        driver.switch_to.window(current_windows[-1])

                    # 스마트 표 찾기 로직
                    try: WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
                    except: pass

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    all_tables = soup.find_all("table")
                    target_tbody = None

                    for table in all_tables:
                        txt = table.get_text(strip=True)
                        if ("제목" in txt and "등록일" in txt) or ("질의요지" in txt) or ("회답내용" in txt):
                            target_tbody = table.find("tbody") or table
                            break

                    if not target_tbody and all_tables:
                        longest = max(all_tables, key=lambda t: len(t.get_text()))
                        target_tbody = longest.find("tbody") or longest

                    # 데이터 저장
                    post_data = {'제목_목록기준': title_text, '페이지': current_browser_page}
                    if target_tbody:
                        for tr in target_tbody.find_all("tr"):
                            th, td = tr.find("th"), tr.find("td")
                            if th and td: post_data[th.get_text(strip=True)] = td.get_text(separator="\n", strip=True)
                            elif not th and len(tr.find_all("td")) >= 2:
                                tds = tr.find_all("td")
                                post_data[tds[0].get_text(strip=True)] = tds[1].get_text(separator="\n", strip=True)
                        results.append(post_data)
                    else:
                        print(" -> [주의] 테이블 못 찾음")

                    # 복귀
                    if is_new_window:
                        driver.close(); driver.switch_to.window(main_window)
                    else:
                        driver.back()
                        try: WebDriverWait(driver, 2).until(EC.alert_is_present()); driver.switch_to.alert.accept()
                        except: pass
                    time.sleep(1)

                except Exception as e:
                    print(f" -> 에러: {e}")
                    try:
                        if len(driver.window_handles) > 1: driver.close(); driver.switch_to.window(main_window)
                        else: driver.back()
                    except: pass
                    continue

            # 중간 저장
            if current_browser_page % 10 == 0:
                pd.DataFrame(results).to_excel(f"./drive/MyDrive/Data/fsc_crawl_temp_{current_browser_page}.xlsx", index=False)
                print(f"--- {current_browser_page}페이지 저장 완료 ---")

            # 다음 페이지 이동
            try:
                next_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginate_button.next")))
                if "disabled" in next_btn.get_attribute("class"):
                    print("마지막 페이지입니다.")
                    break
                driver.execute_script("arguments[0].click();", next_btn)
                current_browser_page += 1
                time.sleep(3)
            except:
                print("다음 페이지 없음.")
                break

    finally:
        if driver: driver.quit()
        if results:
            pd.DataFrame(results).to_excel("fsc_crawl_result_final.xlsx", index=False)
            print("완료")

# 실행 시 시작 페이지 지정
if __name__ == "__main__":
    # 여기 숫자를 40으로 바꾸면 1~39페이지는 건너뛰고 40부터 시작합니다.
    crawl_fsc_from_page(start_page=1141)

사이트 접속 중... (현재 1페이지)
 -> 현재 1페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 2페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 3페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 4페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 5페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 6페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 7페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 8페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 9페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 10페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 11페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 12페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 13페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 14페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 15페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 16페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 17페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 18페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 19페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 20페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 21페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 22페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 23페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 24페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 25페이지... 목표(1141페이지)까지 이동 중...
 -> 현재 26페이지... 목표(1141페이지)까지 이동 중...

### 다른 페이지를 크롤링

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def crawl_fsc_from_page(start_page=1):
    driver = None
    results = []

    try:
        # --- (브라우저 설정 옵션들은 그대로 유지) ---
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--allow-running-insecure-content')
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        options.add_argument(f'user-agent={user_agent}')

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""
        })
        driver.set_page_load_timeout(60)

        # 1. 초기 접속 (무조건 1페이지로 접속됨)
        url = "https://better.fsc.go.kr/fsc_new/replyCase/TotalReplyList.do?stNo=11&muNo=117&muGpNo=75"
        print(f"사이트 접속 중... (현재 1페이지)")
        driver.get(url)
        time.sleep(3)

        # 현재 브라우저의 페이지 상태 (1부터 시작)
        current_browser_page = 1

        while True:
            # ===============================================================
            # [추가된 로직] 목표 페이지(start_page)보다 작으면 무조건 '다음' 클릭해서 넘김
            # ===============================================================
            if current_browser_page < start_page:
                print(f" -> 현재 {current_browser_page}페이지... 목표({start_page}페이지)까지 이동 중...")

                try:
                    # '다음' 버튼 찾기
                    next_btn = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginate_button.next"))
                    )
                    # 다음 버튼 클릭
                    driver.execute_script("arguments[0].click();", next_btn)
                    current_browser_page += 1
                    time.sleep(1) # 이동 대기 (빠르게 넘기기 위해 시간 단축)
                    continue # 수집 로직 건너뛰고 바로 다음 루프로
                except Exception as e:
                    print(f"페이지 이동 중 에러 발생 (더 이상 페이지가 없을 수 있음): {e}")
                    break

            # ===============================================================
            # [수집 로직] 목표 페이지에 도달했을 때부터 수행
            # ===============================================================
            print(f"\n--- {current_browser_page} 페이지 수집 시작 ---")

            try:
                # 목록 로딩 확인
                try:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, "tbody")))
                except:
                    print("목록 로딩 실패.")
                    break

                rows = driver.find_elements(By.CSS_SELECTOR, ".board-list tbody tr")
                # (빈 행 처리 로직 등은 기존과 동일)
                if len(rows) == 0:
                    rows = driver.find_elements(By.TAG_NAME, "tr")
                    rows = [r for r in rows if r.find_elements(By.TAG_NAME, "td")]

                post_count = len(rows)
                print(f"{current_browser_page}페이지: 총 {post_count}개의 게시물 발견")

                if post_count == 0:
                    break

            except Exception as e:
                print(f"목록 에러: {e}")
                break

            # --- (게시물 상세 수집 루프는 기존 코드와 동일하게 유지) ---
            for i in range(post_count):
                try:
                    # Stale 방지 재검색
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "tr")))
                    try: rows = driver.find_elements(By.CSS_SELECTOR, ".board-list tbody tr")
                    except:
                        rows = driver.find_elements(By.TAG_NAME, "tr")
                        rows = [r for r in rows if r.find_elements(By.TAG_NAME, "td")]

                    if i >= len(rows): continue
                    row = rows[i]

                    try:
                        title_link = row.find_element(By.TAG_NAME, "a")
                        title_text = title_link.text
                    except: continue

                    print(f" [{current_browser_page}P - {i+1}/{post_count}] {title_text}")

                    # 윈도우 핸들 저장 및 클릭
                    main_window = driver.current_window_handle
                    driver.execute_script("arguments[0].click();", title_link)
                    time.sleep(2)

                    # 새 창 처리
                    current_windows = driver.window_handles
                    is_new_window = False
                    if len(current_windows) > 1:
                        is_new_window = True
                        driver.switch_to.window(current_windows[-1])

                    # 스마트 표 찾기 로직
                    try: WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
                    except: pass

                    html = driver.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    all_tables = soup.find_all("table")
                    target_tbody = None

                    for table in all_tables:
                        txt = table.get_text(strip=True)
                        if ("제목" in txt and "등록일" in txt) or ("질의요지" in txt) or ("회답내용" in txt):
                            target_tbody = table.find("tbody") or table
                            break

                    if not target_tbody and all_tables:
                        longest = max(all_tables, key=lambda t: len(t.get_text()))
                        target_tbody = longest.find("tbody") or longest

                    # 데이터 저장
                    post_data = {'제목_목록기준': title_text, '페이지': current_browser_page}
                    if target_tbody:
                        for tr in target_tbody.find_all("tr"):
                            th, td = tr.find("th"), tr.find("td")
                            if th and td: post_data[th.get_text(strip=True)] = td.get_text(separator="\n", strip=True)
                            elif not th and len(tr.find_all("td")) >= 2:
                                tds = tr.find_all("td")
                                post_data[tds[0].get_text(strip=True)] = tds[1].get_text(separator="\n", strip=True)
                        results.append(post_data)
                    else:
                        print(" -> [주의] 테이블 못 찾음")

                    # 복귀
                    if is_new_window:
                        driver.close(); driver.switch_to.window(main_window)
                    else:
                        driver.back()
                        try: WebDriverWait(driver, 2).until(EC.alert_is_present()); driver.switch_to.alert.accept()
                        except: pass
                    time.sleep(1)

                except Exception as e:
                    print(f" -> 에러: {e}")
                    try:
                        if len(driver.window_handles) > 1: driver.close(); driver.switch_to.window(main_window)
                        else: driver.back()
                    except: pass
                    continue

            # 중간 저장
            if current_browser_page % 10 == 0:
                pd.DataFrame(results).to_excel(f"./drive/MyDrive/Data/fsc_crawl_temp_{current_browser_page}.xlsx", index=False)
                print(f"--- {current_browser_page}페이지 저장 완료 ---")

            # 다음 페이지 이동
            try:
                next_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginate_button.next")))
                if "disabled" in next_btn.get_attribute("class"):
                    print("마지막 페이지입니다.")
                    break
                driver.execute_script("arguments[0].click();", next_btn)
                current_browser_page += 1
                time.sleep(3)
            except:
                print("다음 페이지 없음.")
                break

    finally:
        if driver: driver.quit()
        if results:
            pd.DataFrame(results).to_excel("fsc_crawl_result_final.xlsx", index=False)
            print("완료")

# 실행 시 시작 페이지 지정
if __name__ == "__main__":
    # 여기 숫자를 40으로 바꾸면 1~39페이지는 건너뛰고 40부터 시작합니다.
    crawl_fsc_from_page(start_page=871)